In [1]:
from urllib2 import urlopen
from bs4 import BeautifulSoup
import csv
import re

In [2]:
base_url = "http://www.trustile.com/"
def make_url(str):
    return base_url + str

In [3]:
def make_soup(url):
    html = urlopen(url).read()
    return BeautifulSoup(html)

In [4]:
soup = make_soup("http://www.trustile.com/products/glass-doors/pl-series")
plList = []
for url in soup.findAll('a'):
    try:
        urlClass = url.get('class')
        if urlClass[0]=="gridview":
            plList.append(url.get('href'))
    except:
        pass
plList


['/products/glass-doors/pl-series/pl100',
 '/products/glass-doors/pl-series/pl104',
 '/products/glass-doors/pl-series/pl106',
 '/products/glass-doors/pl-series/pl109',
 '/products/glass-doors/pl-series/pl119',
 '/products/glass-doors/pl-series/pl130',
 '/products/glass-doors/pl-series/pl132',
 '/products/glass-doors/pl-series/pl134',
 '/products/glass-doors/pl-series/pl136',
 '/products/glass-doors/pl-series/pl143',
 '/products/glass-doors/pl-series/pl144',
 '/products/glass-doors/pl-series/pl145',
 '/products/glass-doors/pl-series/pl146',
 '/products/glass-doors/pl-series/pl200',
 '/products/glass-doors/pl-series/pl202',
 '/products/glass-doors/pl-series/pl204',
 '/products/glass-doors/pl-series/pl206',
 '/products/glass-doors/pl-series/pl209',
 '/products/glass-doors/pl-series/pl219',
 '/products/glass-doors/pl-series/pl220',
 '/products/glass-doors/pl-series/pl224',
 '/products/glass-doors/pl-series/pl226',
 '/products/glass-doors/pl-series/pl229',
 '/products/glass-doors/pl-series/

In [5]:
def get_doorName(soup):
    return soup.h1.string

In [6]:
def get_pageFields(soup):
    return soup.findAll('fieldset')

In [7]:
def get_images(field):
    # get all types of images of this door model
    div = field.find('div',attrs={"class":"doorthumbs"})
    images = []
    for a in div.findAll('a'):
        aImage = {}
        aImage["type"] = a.get('alt')
        aImage["url"] = a.get('href')
        images.append(aImage)
    return images
            

In [8]:
# detail information on the right of the page
def get_details(field):
    div = field.find('div',attrs={"class":"doordetails"})
    details = {}
    for p in div.findAll('p'):
        try:
            ifDetail = p.get('class')[2]
            if ifDetail == "product-detail":           
                category = p.span
                if category.string == u'Series:\xa0':
                    details['Series'] = p.a.string
    
                if category.string == u'Application:\xa0':
                    stri = p.get_text()
                    app = re.search(u'(?<=\xa0).*',stri).group()
                    details['Application'] = app
    
                if category.string == u'Thickness:\xa0':
                    details['Thickness'] = p.findAll('span')[1].get_text()
    
                if category.string == u'Construction:\xa0':
                    try:
                        construction = []
                        for child in p.findAll('span')[1].children:
                            construction.append(child.string)
                    except:
                        constrction = re.search(u'(?<=\xa0).*',p.get_text()).group()
                    details['Construction'] = construction
                if category.string == u'Available Fire Rating:\xa0':
                    fireRating = []
                    for img in p.findAll('span')[1].findAll('img'):
                        aRating = {}
                        aRating["type"] = img.get('alt')
                        aRating["url"] = img.get('src')
                        fireRating.append(aRating)
                    details['Fire'] = fireRating
        except:
            print p.get_text()
    return details

In [25]:

def get_profiles(field):
    
    headers = field.findAll('h3')
    tables = field.findAll('table')
    
    profiles = []
    
    for i in range(len(headers)):
#         print headers[i].string
#         print "=========="
        aProfileType = {}
        aProfileType["type"] = headers[i].string
        options = []
        for td in tables[i].findAll('td'):
            try:
                a = td.a
                label = a.get('title')
            
                if headers[i].string == "Sash Options":
                    options.append(label)
                elif headers[i].string == "Panel Options":
                    ab = re.search(u'(?<=\u201c).(?=\u201d)',label).group()
                    options.append(ab)
                else:
                    ab = re.search(u'(?<=\().*(?=\))',label).group()
                    options.append(ab)
            except:
                pass
            aProfileType["options"] = options
        profiles.append(aProfileType)
    return profiles

In [28]:
# PL series
# 
outFile = open("TrustilePL_Doors.csv", "wb")
writer = csv.writer(outFile)
writer.writerow(["","Name","Images","Series","Application","Construction","Thickness","Fire Rating","Profiles Options","Other Options"])

for i in range(len(plList)):
    
    soup = make_soup(make_url(plList[i]))
    
    name = get_doorName(soup)
    
    options = []
    fields = get_pageFields(soup)
    for field in fields:
        if field.legend.span.string == "Overview":
            images = get_images(field)
            
            details = get_details(field)
            try:
                fireRating = details["Fire"]
            except:
                fireRating = ""
        
        
        if field.legend.span.string == "Profiles":
            profiles = get_profiles(field)
#             print profiles
        
        # tab3: wood species
        if field.legend.span.string == "Wood Species":
            options.append('Wood')
    
        # tab4: glass options
        if field.legend.span.string == "Glass":
            options.append('Glass')
        
        # tab5: leather options
        if field.legend.span.string == "Leather":
            options.append('Leather')
    
        # tab6: Metal options
        if field.legend.span.string == "Metal":
            options.append('Metal')
    
    writer.writerow([i+1, name, images, 
                     details["Series"], details["Application"], details["Construction"], details["Thickness"], fireRating,
                     profiles,
                     options])
outFile.close()